* Many programs in computer science are written to optimize some value; 
* 컴퓨터 과학의 많은 프로그램들은 값을 최적화 하기 위해 사용된다.
* for example, find the shortest path between two points, find the line that best fits a set of points, or find the smallest set of objects that satisfies some criteria. 
* 예를들어, 두 점의 가장 짧은 경로를 찾는다거나, 점들을 가장 잘 설명해주는 선을 찾아준다거나, 조건을 만족하는 가장 작은 오브젝트 셋들을 찾는 것이다.
* There are many strategies that computer scientists use to solve these problems. 
* 위 문제를 위해 컴퓨터 과학자들이 사용하는 방법은 매우 많이 있다.
* One of the goals of this book is to expose you to several different problem solving strategies. 
* 이 책의 목표중 하나는 문제를 해결하기 위해 많은 방법을 설명해 주는 것이다.
* Dynamic programming is one strategy for these types of optimization problems.
* 동적 프로그래밍 또한 이런 최적화 문제를 해결하는데 하나의 전략이다.

* A classic example of an optimization problem involves making change using the fewest coins. 
* 최적화 문제의 고전적인 예제는 가장 적게 동전을 사용하여 거스름돈 주기이다.
* Suppose you are a programmer for a vending machine manufacturer. 
* 만약 당신이 자판이회사의 프로그래머라고 생각해 보자
* Your company wants to streamline effort by giving out the fewest possible coins in change for each transaction. 
* 회사는 각 거래마다 효율적으로 가장 적은 동전을 사용해 거스름돈을 주고싶어한다.
* Suppose a customer puts in a dollar bill and purchases an item for 37 cents. 
* 만약 고객이 1달러를 넣고 37센트짜리 물건을 샀다고 생각해보자.
* What is the smallest number of coins you can use to make change?
* 몇개의 동전을 사용하면 잔돈을 거슬러줄때 가장 적은 동전을 쓸수 있을까?
* The answer is six coins: two quarters, one dime, and three pennies. 
* 정답은 6개 코인이다. 25센트 2개, 10센트하나, 1센트 3개이다.
* How did we arrive at the answer of six coins? 
* 어떻게 이 정답에 도달했는가?
* We start with the largest coin in our arsenal (a quarter) and use as many of those as possible, then we go to the next lowest coin value and use as many of those as possible. 
* 먼저 우리는 가장 큰 동전부터 시작해 최대한 많이 사용하고, 다음 작은 가치의 동전을 많이 사용하는 방식을 이용한다.
* This first approach is called a greedy method because we try to solve as big a piece of the problem as possible right away.
* 이 첫번쨰 방식을 greedy 메소드라고 하는데 왜냐하면 이 문제를 해결할때 문제의 큰 문제를 바로 해결하려고 하기 때문이다.

* The greedy method works fine when we are using U.S. coins, but suppose that your company decides to deploy its vending machines in Lower Elbonia where, in addition to the usual 1, 5, 10, and 25 cent coins they also have a 21 cent coin.
* 이 greedy 메소드는 미국 동전을 대상으로 할때는 잘 먹히지만, 회사가 자판기를 Lower Elbonia에 설치한다고 가정해보자. 거기는 1,5,10,25센트 코인도 있지만 21센트 코인도 있다.
* In this instance our greedy method fails to find the optimal solution for 63 cents in change.
* 이 경우에는 우리의 greedy 메소드가 63센트를 거슬러줄때 최적화에 실패할 것이다.
* With the addition of the 21 cent coin the greedy method would still find the solution to be six coins. 
* 21센트 코인을 추가함에도, greedy 메소드는 여전히 6개 동전을 줄것이다.
* However, the optimal answer is three 21 cent pieces.
* 그러나 문제의 최적값은 21센트 동전 3개이다.

* Let’s look at a method where we could be sure that we would find the optimal answer to the problem. 
* 문제에 대해 최적의 답을 찾을수 있는 방법을 살펴보자.
* Since this section is about recursion, you may have guessed that we will use a recursive solution. 
* 이 섹션이 재귀에 관련한 섹션이기 때문에, 재귀문제로 풀어볼것이라 추측할 수 있다.
* Let’s start with identifying the base case. 
* base case를 확인하는 것으로 시작해보자.
* If we are trying to make change for the same amount as the value of one of our coins, the answer is easy, one coin.
* 만약 우리가 잔돈을 동일한 가치의 동전으로 가지고 있다면 답은 쉽다, 코인 1개이다.

* If the amount does not match we have several options. 
* 만약 이 값이 맞지 않는다면 우리는 다양한 옵션을 가지고 있다.
* What we want is the minimum of a penny plus the number of coins needed to make change for the original amount minus a penny, or a nickel plus the number of coins needed to make change for the original amount minus five cents, or a dime plus the number of coins needed to make change for the original amount minus ten cents, and so on. 
* 우리가 원하는것은 원래 금액에서 1페니를 빼고 필요한 만큼의 동전수를 더하거나, 원래 금액에서 5센트를 빼고 필요한 만큼의 동전수를 더하거나, 10센트, 25센트를 빼고 필요한 만큼의 동전 수를 더하는 것이다.
* So the number of coins needed to make change for the original amount can be computed according to the following:
* 그래서 우리가 잔돈을 거슬러 주기 위한 동전의 숫자를 계산하는공식은 아래와 같다

numCoins=min(1+numCoins(originalamount−1)1+numCoins(originalamount−5)1+numCoins(originalamount−10)1+numCoins(originalamount−25))

* The algorithm for doing what we have just described is shown in Listing 7. 
* 이것을 계산하기 위한 알고리즘은 리스팅 7에 나와있다. 
* In line 3 we are checking our base case; that is, we are trying to make change in the exact amount of one of our coins. 
* 3라인에서 우리의 base case를 체크한다. 그것은 우리의 코인중 하나와 잔돈이 똑같은 것이다.
* If we do not have a coin equal to the amount of change, we make recursive calls for each different coin value less than the amount of change we are trying to make. 
* 만약 우리가 잔돈과 가지고 있는 코인이 같지 않다면, 각각의 코인 가치와 맞는 재귀 호출을 하게 된다.
* Line 6 shows how we filter the list of coins to those less than the current value of change using a list comprehension.
* 6라인에서는 list comprehension을 사용해 잔돈보다 작은 코인들의 리스트를 만드는것을 보여준다.
* The recursive call also reduces the total amount of change we need to make by the value of the coin selected.
* 재귀 호출은 선택한 코인의 가치에 따라 잔돈의 양을 줄여준다.
* The recursive call is made in line 7. 
* 이 재귀 호출은 7 라인에서 일어난다.
* Notice that on that same line we add 1 to our number of coins to account for the fact that we are using a coin. 
* 우리가 코인을 사용할때 numCoin에 1을 더해주는것에 유의해라
* Just adding 1 is the same as if we had made a recursive call asking where we satisfy the base case condition immediately.
* 1을 더하는 것은 base condition을 충족시키는 것인지 확인하는것과 같다.

In [1]:
# Listing 7

def recMC(coinValueList,change):
    minCoins = change
    if change in coinValueList:
        return 1
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recMC(coinValueList,change-i)
            if numCoins < minCoins:
                minCoins = numCoins
    return minCoins

print(recMC([1,5,10,25],63))

6


* The trouble with the algorithm in Listing 7 is that it is extremely inefficient. 
* 리스팅7 알고리즘의 문제점은 이게 극히 비효율적이라는 것이다.
* In fact, it takes 67,716,925 recursive calls to find the optimal solution to the 4 coins, 63 cents problem! 
* 사실, 4가지 코인으로 63센트 거슬러주기 문제를 이것은 67백만 번정도의 재귀호출을 통해 해결한다.
* To understand the fatal flaw in our approach look at Figure 5, which illustrates a small fraction of the 377 function calls needed to find the optimal set of coins to make change for 26 cents.
* 이것의 결정적인 흠을 이해하기 위해 figure 5를 확인해 보자. 이 그림은 26센트를 거슬러 주기 위해 377번의 재귀 호출이 필요하다.

* Each node in the graph corresponds to a call to recMC. 
* 그래프 안의 각 노드들은 recMc를 call 하는 것에 대응한다.
* The label on the node indicates the amount of change for which we are computing the number of coins. 
* 노드의 라벨은 잔돈의 양을 나타낸다.
* The label on the arrow indicates the coin that we just used. 
* 화살표의 라벨은 우리가 사용한 코인을 나타낸다.
* By following the graph we can see the combination of coins that got us to any point in the graph. 
* 그래프를 따라가면, 우리가 사용한 코인의 조합을 볼 수 있다.
* The main problem is that we are re-doing too many calculations.
* 가장 큰 문제는 우리가 너무 많은 계산을 반복하고 있다는 점이다.
* For example, the graph shows that the algorithm would recalculate the optimal number of coins to make change for 15 cents at least three times.
* 예를 들어, 그래프는 15센트를 계산하기 위해 최소 3번은 계산해야 한다.
* Each of these computations to find the optimal number of coins for 15 cents itself takes 52 function calls. 
* 각각의 계산을 통해 코인의 최적 수를 계산하기 위해서는 52번의 펑션 호출이 필요하다.
* Clearly we are wasting a lot of time and effort recalculating old results.
* 명백히 이것은 이전 결과물들을 계산하기 위해 많은 시간과 노력을 낭비하고 있는것을 보여준다.

* The key to cutting down on the amount of work we do is to remember some of the past results so we can avoid recomputing results we already know. 
* 이런 우리가 이미 알고있는 결과를 다시 계산하는것 같이 쓸데없는 일을 줄이기위해 할수 있는 방법들이 있다.
* A simple solution is to store the results for the minimum number of coins in a table when we find them. 
* 간단한 해결책은 우리가 최소 코인의 수를 계산해낸다면 그 값을 저장해 두는 것이다.
* Then before we compute a new minimum, we first check the table to see if a result is already known. 
* 그러면 우리는 먼저 이 결과값을 비교해 체크할수 있을 것이다.
* If there is already a result in the table, we use the value from the table rather than recomputing. 
* 만약 이 결과값을 이미 갖고 있다면, 다시 계산하는 것 보다 이 값을 사용할 것이다.
* ActiveCode 1 shows a modified algorithm to incorporate our table lookup scheme.
* ActiveCode 1는 테이블을 살펴보는 수정된 알고리즘을 사용한다

In [2]:
def recDC(coinValueList,change,knownResults):
    minCoins = change
    if change in coinValueList:
        knownResults[change] = 1
        return 1
    elif knownResults[change] > 0:
        return knownResults[change]
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1 + recDC(coinValueList, change-i, knownResults)
            if numCoins < minCoins:
                minCoins = numCoins
                knownResults[change] = minCoins
    return minCoins

print(recDC([1,5,10,25],63,[0]*64))

6


* Notice that in line 6 we have added a test to see if our table contains the minimum number of coins for a certain amount of change. 
* 6 라인에서 잔돈의 값에 따라 필요한 최소 코인의 양을 저장해 놓는것을 확인해라.
* If it does not, we compute the minimum recursively and store the computed minimum in the table. 
* 만약 최소 코인의 양이 없다면, 재귀 호출을 최소화하여 최소값을 미리 테이블에 저장해 둘 것이다.
* Using this modified algorithm reduces the number of recursive calls we need to make for the four coin, 63 cent problem to 221 calls!
* 이 수정된 알고리즘을 사용한다면, 4가지 코인을 사용해 63센트 문제를 해결하는것은 221번만 호출하면 된다.

* Although the algorithm in AcitveCode 1 is correct, it looks and feels like a bit of a hack. 
* 비록 ActiveCode1의 알고리즘이 정확하지만, 약간 복잡하고 어려워 보인다.
* Also, if we look at the knownResults lists we can see that there are some holes in the table. 
* 또한, knownResults 리스트를 살펴보면 살펴보면 테이블에 구멍이 보이는것을 확인할 수 있다.
* In fact the term for what we have done is not dynamic programming but rather we have improved the performance of our program by using a technique known as “memoization,” or more commonly called “caching.”
* 사실 우리가 한것은 동적 프로그래밍이 아니라, memoization 혹은 캐싱이라고 불리는 테크닉을 사용하여 프로그램의 성능을 향상시킨 것이다.

* A truly dynamic programming algorithm will take a more systematic approach to the problem. 
* 진짜 동적 프로그래밍 알고리즘은 문제에 대해 좀더 체계적인 접근을 사용한다.
* Our dynamic programming solution is going to start with making change for one cent and systematically work its way up to the amount of change we require. 
* 우리의 동적 프로그래밍 해결책은 1센트 부터 시작해 우리가 필요한 잔돈의 양까지 체계적으로 계산할 것이다.
* This guarantees us that at each step of the algorithm we already know the minimum number of coins needed to make change for any smaller amount.
* 이것은 각 단계의 알고리즘에서 더 작은 금액을 변경하는데 필요한 코인을 수를 미리 알고 있는 것을 보증하는 것이다.

* Let’s look at how we would fill in a table of minimum coins to use in making change for 11 cents.
* 11센트 거스름돈을 주는데 필요한 최소 코인을 테이블에 기록하는 것을 보자.
* Figure 4 illustrates the process. 
* Figure 4가 프로세스를 보여주고 있다.
* We start with one cent. 
* 1센트로 시작하자
* The only solution possible is one coin (a penny).
* 유일한 해결책은 1페니 동전 하나이다.
* The next row shows the minimum for one cent and two cents.
* 다음행은 1센트와 2센트의 최소값을 보여준다.
* Again, the only solution is two pennies. 
* 마찬가지로 해결책은 1페니 동전 2개이다.
* The fifth row is where things get interesting. 
* 이제 5번째 행이 재밌어지는 부분이다.
* Now we have two options to consider, five pennies or one nickel. 
* 이제 우리는 두가지 옵션을 놓고 고민할 수 있다. 1센트 1개와 5센트 하나.
* How do we decide which is best? We consult the table and see that the number of coins needed to make change for four cents is four, plus one more penny to make five, equals five coins. 
* 어떤것이 최적이라고 결정할수 있을까? 우리가 테이블을 만들고, 4센트 잔돈을 거슬러주는데는 4개가 필요하고, 1페니를 추가하면 5개이므로 5개 동전이다.
* Or we can look at zero cents plus one more nickel to make five cents equals 1 coin. Since the minimum of one and five is one we store 1 in the table.
* 아니면 0 센트와 1개의 5센트 동전은 1개 동전이 필요하다. 1과 5중에 최소값은 1이기 때문에, 1을 테이블에 기록한다.
* Fast forward again to the end of the table and consider 11 cents. 
* 테이블의 끝으로 돌아가 11센트를 놓고 생각해보자
* Figure 5 shows the three options that we have to consider:
* Figure 5는 우리가 생각할 수 있는 세가지 옵션을 보여준다.

* A penny plus the minimum number of coins to make change for 11−1=10 cents (1)
* 페니 + 10센트를 거슬러주기 위한 최소 동전 1
* A nickel plus the minimum number of coins to make change for 11−5=6 cents (2)
* 5센트 + 6센트를 거슬러주기 위한 최소 동전 2
* A dime plus the minimum number of coins to make change for 11−10=1 cent (1)
* 10센트 + 1센트를 거슬러 주기 위한 최소 동전 1

* Either option 1 or 3 will give us a total of two coins which is the minimum number of coins for 11 cents.
* 1~3의 옵션중에 11을 거슬러 주기 위한 최소 동전의 수는 2이다

* Listing 8 is a dynamic programming algorithm to solve our change-making problem. 
* Listing 8은 동적 프로그래밍 알고리즘을 사용해 잔돈을 거슬러주는 문제를 해결한 것이다.
* dpMakeChange takes three parameters: a list of valid coin values, the amount of change we want to make, and a list of the minimum number of coins needed to make each value. 
* dpMakeChange는 세개의 파라미터를 받는다, 유효한 동전의 값들, 거슬러주기 원하는 잔돈의 양, 그리고 각 값의 최소 코인의 수 이다.
* When the function is done minCoins will contain the solution for all values from 0 to the value of change.
* function이 종료되면 minCoins는 0부터 값까지에 대한 결과값을 가지고 있다.

In [5]:
# Listing 8

def dpMakeChange(coinValueList,change,minCoins):
    for cents in range(change+1):
        coinCount = cents
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
        minCoins[cents] = coinCount
    return minCoins[change]

* Note that dpMakeChange is not a recursive function, even though we started with a recursive solution to this problem. 
* 우리가 이 문제를 해결하기 위해 재귀 호출로 출발 했지만, dpMakeChange는 재귀호출 함수가 아니라는 것을 주목하라
* It is important to realize that just because you can write a recursive solution to a problem does not mean it is the best or most efficient solution.
* 단순하게 재귀 호출로 문제를 해결하는 것이 문제를 해결하는데 최적이나 최고의 방법이 아니라는것을 깨닫는것이 중요하다.
* The bulk of the work in this function is done by the loop that starts on line 4. 
* 이 펑션의 작업을 하는것은 4라인에서 시작하는 반복문이다.
* In this loop we consider using all possible coins to make change for the amount specified by cents. 
* 이 반복문에서 모든 가능한 수의 잔돈에 따른 필요한 최소 동전을 저장해둔다.
* Like we did for the 11 cent example above, we remember the minimum value and store it in our minCoins list.
* 위 예제에서 11센트를 계산한 것 처럼 minCoins 리스트에 최소 값을 저장해둔다.

* Although our making change algorithm does a good job of figuring out the minimum number of coins, it does not help us make change since we do not keep track of the coins we use. 
* 비록 우리의 잔돈 거슬러주는 알고리즘이 최소 동전의 갯수를 알아내는데는 도움을 주지만, 어떤 동전을 사용하는지는 기록하지 않는다.
* We can easily extend dpMakeChange to keep track of the coins used by simply remembering the last coin we add for each entry in the minCoins table.
* 이것은 dpMakeChange를 쉽게 extends하여 minCoins table에 마지막엔 어떤 동전을 사용했는지 기록해 주면 된다
* If we know the last coin added, we can simply subtract the value of the coin to find a previous entry in the table that tells us the last coin we added to make that amount. 
* 만약 우리가 마지막에 사용한 코인을 기록하면 이전단계의 테이블을 찾아서 어떤 코인들을 사용했는지 알 수 있다.
* We can keep tracing back through the table until we get to the beginning.
* 테이블을 계속 추적하면 처음 값부터 모두 사용한 코인을 알수 있을 것이다.

* ActiveCode 2 shows the dpMakeChange algorithm modified to keep track of the coins used, along with a function printCoins that walks backward through the table to print out the value of each coin used. 
* ActiveCode 2는 dpMakeChange 변경한 알고리즘을 사용해 사용한 코인을 기록하고 printCoins를 호출해 각각 어떤 코인을 사용했는지 역순으로 알려준다. 
* This shows the algorithm in action solving the problem for our friends in Lower Elbonia. 
* 이것은 Lower Elbonia에 있는 친구들에게 문제 해결하는 방법도 알려준다.
* The first two lines of main set the amount to be converted and create the list of coins used. The next two lines create the lists we need to store the results.
* 처음 main에 있는 처음 두 라인이 거스름돈의 양을 세팅하고, 어떤 코인을 사용할수 있는지 보여준다.
* coinsUsed is a list of the coins used to make change, and coinCount is the minimum number of coins used to make change for the amount corresponding to the position in the list.
* coinsUsed는 이 잔돈을 만들떄 어떤 동전을 사용했는지, 그리고 coinCount는 리스트에 있는 최소 코인의 수를 나타내 준다.

* Notice that the coins we print out come directly from the coinsUsed array. 
* 우리가 출력하는 코인들은 coinsUsed array에서 바로 출력한 것을 알아두어라.
* For the first call we start at array position 63 and print 21. 
* 처음 호출하는 부분은 array의 63위치에서 21을 출력하는 것이다.
* Then we take 63−21=42 and look at the 42nd element of the list.
* 그리고 리스트의 63-21인 42번째 엘리먼트를 출력한다.

* Once again we find a 21 stored there. 
* 다시 한번 여기서 21이 저장되어있다.
* Finally, element 21 of the array also contains 21, giving us the three 21 cent pieces.
* 마지막으로, 21번째 어레이의 엘리먼트도 21이므로 21센트가 세번 출력된다.

In [4]:
def dpMakeChange(coinValueList,change,minCoins,coinsUsed):
    for cents in range(change+1):
        coinCount = cents
        newCoin = 1
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents-j] + 1 < coinCount:
                coinCount = minCoins[cents-j]+1
                newCoin = j
        minCoins[cents] = coinCount
        coinsUsed[cents] = newCoin
    return minCoins[change]

def printCoins(coinsUsed,change):
    coin = change
    while coin > 0:
        thisCoin = coinsUsed[coin]
        print(thisCoin)
        coin = coin - thisCoin

def main():
    amnt = 63
    clist = [1,5,10,21,25]
    coinsUsed = [0]*(amnt+1)
    coinCount = [0]*(amnt+1)

    print("Making change for",amnt,"requires")
    print(dpMakeChange(clist,amnt,coinCount,coinsUsed),"coins")
    print("They are:")
    printCoins(coinsUsed,amnt)
    print("The used list is as follows:")
    print(coinsUsed)

main()

Making change for 63 requires
3 coins
They are:
21
21
21
The used list is as follows:
[1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 21, 1, 1, 1, 25, 1, 1, 1, 1, 5, 10, 1, 1, 1, 10, 1, 1, 1, 1, 5, 10, 21, 1, 1, 10, 21, 1, 1, 1, 25, 1, 10, 1, 1, 5, 10, 1, 1, 1, 10, 1, 10, 21]
